In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from ilxutils.ttl2pd import *
import difflib

def degrade(var):
    def helper(s):
        return str(re.sub("\(|\)|'|,", "", s).lower().strip())
    if type(var)!=list:
        if var:
            return helper(var)
        else:
            return var
    else:
        return [helper(v) if v else v for v in var]
    
def cj(data, output):
    with open(output+'.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print('Complete')
def oj(filepath):
    with open(filepath+'.pickle', 'r') as infile:
        return json.load(infile)
        
def cp(data, output):
    with open(output+'.pickle', 'wb') as outfile:
        pickle.dump(data, outfile, indent=4)
    print('Complete')
def op(filepath):
    with open(filepath+'.pickle', 'rb') as infile:
        return pickle.load(infile)
        
args = read_args(api_key= p.home() / 'keys/production_api_scicrunch_key.txt', db_url= p.home() / 'keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [2]:
terms = sql.get_terms()
annos = sql.get_annotations()

In [4]:
terms[terms.id == 182]

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
181,182,ilx_0100181,32290,0,32290,0,Abnormal Involuntary Movement Scale,The AIMS is an assessment of movement dysfunct...,,term,4,0,1,1481227063,1521840805


In [3]:
annos.head()

,id,item_id,label,ilx,anno_tid,anno_label,anno_ilx,tid,annotation_tid,value
0,38201,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,categories : Questionnaire
1,38202,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,status : Published
2,38208,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,dataType : Clinical Assessments
3,38216,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,sources : NDAR; NDCT; RDoC
4,38220,182,Abnormal Involuntary Movement Scale,ilx_0100181,15074,hasConstraint,ilx_0115071,182,15074,modifiedDate : 1515012631513


In [4]:
stak = { #second tier annotation keys
    'condition':True,
    'filterElement':True,
    'id':True,
    'notes':True,
    'required':True,
    'size':True,
    'type':True,
    'valueRange':True,
}
def contains_key(string):
    if ':' in string:
        sk, sval = string.split(':', 1)[0].strip(), string.split(':', 1)[1].strip() #string key
        if stak.get(sk):
            return sk, sval
    return False, False

In [ ]:
cp(annos, '../../../ilx_db_anno_backup')

In [ ]:
op('../../../ilx_db_anno_backup')

In [ ]:
udata = []
adata = []
cdata = []
for i, row in annos[annos.anno_label == 'hasConstraint'].iterrows(): ## get second tier annotations only -> anno_label == 'hasConstraint'
    key_type, key_val = contains_key(row.value): ## filter each term by their annotation values by key : value
    
    if key_type == 'condition':
        cdata.append({'codition':key_val})
    
    elif key_type == 'filterElement':
        cdata.append({'filterElement':key_val})
    
    elif key_type == 'id':
        pass#udata.append({**row, **{'delete':True}}) ## maybe have to to_dict row
    
    elif key_type == 'notes':
        pass#adata.append({**row, **{'notes':key_val}})
        #udata.append({**row, **{'delete':True}})
    
    elif key_type == 'required':
        cdata.append({'required':key_val})
    
    elif key_type == 'size':
        cdata.append({'size':key_val})
    
    elif key_type == 'type':
        cdata.append({'type':key_val})
    
    elif key_type == 'valueRange':
        pass#minValue, maxValue = key_val.split('::')[0].split(), key_val.split('::')[1].split()
        #udata.append({**row, **{'delete':True}})
        #adata.append({**row, **{'value':minValue}})
        #adata.append({**row, **{'value':maxValue}})
        
size_examples = set([v for k,v in data.items() if k=='size'])
print(size_examples)
cj('../../../nda_annotation_currections')

## condition -> ?
## filterElement -> ?
## delete Id if it is preferred already
## notes -> move to notes
## required -> get list of options this has
## size -> get ex that isn't none
## type -> porb
## valueRange -> seperate into minValue and maxValue